In [606]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
import warnings
import psycopg2
import re

In [607]:
#스크롤 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,6000);")
    time.sleep(2)

# 해시태그 함수
def get_hashtag(outline_test) :
    # hash_tag가 안달려있는 영상이 있기때문에 오류 방지
    try:
        hash_tag = " ".join(re.findall("#[가-힣]{1,}", outline_test))
        print(hash_tag)
        hashtag_list.append(hash_tag)
            
    except Exception as e:
        hash_tag = ""
        print(hash_tag)
        hashtag_list.append(hash_tag)

#select문 만들기
def select_command(table) :
    command = 'select * from ' + table
    return command

#insert문 만들기
def insert_command(table) :
    command = 'insert into ' + table + ' (title, date, url, hashtag) values (%s, %s, %s, %s);'
    return command

뷔스티에 원피스, 롱패딩, 숏패딩, 롱코트, 숏코트, 잠옷, 홈웨어, 트위드자켓, 블레이저 자켓, 싱글 자켓, 크롭 자켓, 레더 자켓, 루즈핏, 테일러드 자켓, 린넨자켓, 반팔자켓, 라이더자켓, 오버핏자켓, 청자켓, 니트가디건, 볼레로, 아가일, 크롭가디건, 반팔가디건', '앙고라', '바람막이', '아노락', '윈드브레이커', '바시티자켓', '바시티점퍼', '야구점퍼', 후드집업, 투웨이집업, 여름아우터, 간절기아우터, 겨울아우터, 기모맨투맨, 니트맨투맨, 크롭 맨투맨', '남자 맨투맨', '반집업', 후드티, 남자후드, 꾸안꾸, 플리스, 반팔티, 크롭티, 블라우스, 와이셔츠, 셔츠, 체크셔츠, 타이다이, 옥스퍼드셔츠, 남방, 슬리브티, 니트, 남자니트, 조끼, '봄슬랙스', '여름슬랙스', '가을슬랙스', '겨울슬랙스', '간절기슬랙스', '데님팬츠', '연청', '진청', '중청', '흑청', '그레이진', '와이드팬츠', '밴딩바지', '스키니진', '린넨팬츠', '반바지', '하이웨스트', '로우웨스트', '부츠컷', '치노팬츠', '트레이닝복', '트레이닝바지', '조거팬츠', '카고바지', '일자팬츠', '냉장고바지', '멜빵바지', '오버롤팬츠', '찢청', 

keyword_list = ['미니스커트', '미디스커트', '롱스커트', '플레어스커트', 'A라인 스커트', 'H라인 스커트', '청치마', '테니스스커트', '프릴스커트', '레더 스커트', '랩스커트', '데님스커트', '플리츠스커트', '미니원피스', '미디원피스', '롱원피스', '나시원피스', '코르셋', '셔츠원피스', '벨벳원피스', '꽃원피스', '골지원피스', '골지티', '쉬폰원피스', '쉬폰블라우스', '린넨원피스', '카라원피스', '랩원피스', '니트원피스', '후드원피스', '코듀로이', '반팔원피스', '점프슈트', '봄룩북', '봄옷쇼핑', '봄코디', '봄옷하울', '여름룩북', '여름옷쇼핑', '여름코디', '여름하울', '가을룩북', '가을옷쇼핑', '가을코디', '가을하울']

In [609]:
warnings.filterwarnings(action='ignore')

driver = webdriver.Chrome('C:\chromedriver.exe')

# 키워드를 검색한 유튜브 주소 접속
keyword = '테니스스커트'
url = 'https://www.youtube.com/results?search_query={}'.format(keyword)
driver.get(url)

#사이트 최대화
driver.maximize_window()

# 필터 클릭
driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/div/ytd-toggle-button-renderer/a/tp-yt-paper-button').click()
time.sleep(1)

# 구분 -> 동영상 클릭
driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[2]/ytd-search-filter-renderer[1]/a/div/yt-formatted-string').click()

# 업로드 날짜 -> 올해 클릭
driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[1]/ytd-search-filter-renderer[5]/a').click()

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 100
Current browser version is 102.0.5005.63 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	Ordinal0 [0x00807413+2389011]
	Ordinal0 [0x00799F61+1941345]
	Ordinal0 [0x0068C658+837208]
	Ordinal0 [0x006AC773+968563]
	Ordinal0 [0x006A837A+951162]
	Ordinal0 [0x006A5C51+941137]
	Ordinal0 [0x006D8C80+1150080]
	Ordinal0 [0x006D88DA+1149146]
	Ordinal0 [0x006D3F66+1130342]
	Ordinal0 [0x006AE546+976198]
	Ordinal0 [0x006AF456+980054]
	GetHandleVerifier [0x009B9632+1727522]
	GetHandleVerifier [0x00A6BA4D+2457661]
	GetHandleVerifier [0x0089EB81+569713]
	GetHandleVerifier [0x0089DD76+566118]
	Ordinal0 [0x007A0B2B+1968939]
	Ordinal0 [0x007A5988+1989000]
	Ordinal0 [0x007A5A75+1989237]
	Ordinal0 [0x007AECB1+2026673]
	BaseThreadInitThunk [0x76A4FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77A87A7E+286]
	RtlGetAppContainerNamedObjectPath [0x7

In [599]:
# 제목 & 링크 크롤링

# 스크롤 다운 50번 실행
body = driver.find_element_by_tag_name('body')
body.send_keys(Keys.PAGE_DOWN)

for i in range(1,50):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

soup = BeautifulSoup(driver.page_source, 'html.parser')
name = soup.select('a#video-title')
video_url = soup.select('a#video-title')

video_num_list = [] #영상번호
name_list = [] #영상제목
url_list = [] #영상주소

for i in range(len(name)):
    video_num_list.append(i+1)
    name_list.append(name[i].text.strip())

for i in video_url:
    url_list.append('{}{}'.format('https://www.youtube.com',i.get('href')))

print('영상 개수: ', len(name_list))

영상 개수:  153


In [601]:
# 업로드일 & 해시태그 가져오기
day_list = [] #업로드일
hashtag_list = [] #해시태그

for i in range(0,len(name_list)):
    
    if url_list[i].split('/')[3] == 'shorts':  #쇼츠

        print(i+1, ': 쇼츠', end=' ')

        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
        
        driver.get(url_list[i])

        time.sleep(2)

        #더보기 클릭
        driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-shorts/div[1]/ytd-reel-video-renderer[1]/div[2]/ytd-reel-player-overlay-renderer/div[2]/div[1]/ytd-menu-renderer/yt-icon-button/button/yt-icon').click()
        time.sleep(2)

        #설명 클릭
        driver.find_element_by_xpath('/html/body/ytd-app/ytd-popup-container/tp-yt-iron-dropdown/div/ytd-menu-popup-renderer/tp-yt-paper-listbox/ytd-menu-service-item-renderer/tp-yt-paper-item').click()
        time.sleep(2)

        #업로드일 정보 가져오기
        day = driver.find_element_by_id("publish-time").text.split(':')[1].strip()

        print(day)
        day_list.append(day)

        time.sleep(2) 

        # 해시태그 가져오기
        outline_test = driver.find_element_by_class_name("style-scope ytd-reel-description-sheet-renderer").text
        get_hashtag(outline_test)

    else :
        print(i+1, ': 일반', end=' ')

        driver.get(url_list[i])
            
        time.sleep(2)    

        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')

        time.sleep(2)  

        #업로드일 정보 가져오기
        day = soup.select('#formatted-snippet-text > span:nth-child(3)')[0].text
        #day = driver.find_element_by_id("info-strings").text.split('\n')[0]
        
        print(day)
        day_list.append(day)

        time.sleep(2) 

        # 해시태그 가져오기
        outline_test = soup.find("yt-formatted-string", {"class":"content style-scope ytd-video-secondary-info-renderer"}).text    
        get_hashtag(outline_test)

print('끝')

1 : 일반 2022. 5. 10.

2 : 쇼츠 2021. 11. 19.

3 : 일반 2022. 1. 25.
#코디 #겨울 #옷
4 : 쇼츠 2022. 2. 5.

5 : 쇼츠 2022. 2. 1.

6 : 일반 2021. 6. 4.
#청치마코디 #이직뚜 #여름코디여자
7 : 쇼츠 2022. 1. 25.

8 : 일반 2022. 1. 11.
#코디 #겨울 #옷
9 : 쇼츠 2021. 12. 11.

10 : 일반 2021. 12. 22.
#체형 #체형별바지 #체형별코디 #체형교정
11 : 일반 2022. 1. 12.
#롱치마 #롱스커트
12 : 쇼츠 2022. 3. 18.

13 : 일반 2022. 3. 17.

14 : 일반 2022. 2. 27.

15 : 일반 2022. 3. 13.
#개강룩 #대학생코디 #데일리룩
16 : 쇼츠 2021. 10. 30.

17 : 일반 2021. 6. 18.
#무신사 #무탠우먼즈 #패션디자이너 #디자이너 #옷 #스타일링 #코디 #스타일리스트 #코디네이터 #청바지 #청치마 #데님 #찢청 #청청패션
18 : 쇼츠 2021. 12. 28.

19 : 일반 2022. 5. 6.
#여자중학생 #여자학생코디 #중학생옷
20 : 쇼츠 2021. 11. 1.

21 : 쇼츠 2021. 7. 24.

22 : 일반 2021. 6. 30.
#테니스스커트 #주름스커트
23 : 쇼츠 2022. 5. 10.

24 : 쇼츠 2021. 10. 12.

25 : 쇼츠 2021. 11. 2.

26 : 쇼츠 2021. 8. 8.

27 : 일반 2022. 3. 27.

28 : 쇼츠 2021. 8. 26.
#데일리룩 #데이트룩 #키치룰 #캠퍼스룩 #비바소피 #룩북 #여자가을코디 #패션하울 #패션룩북
29 : 쇼츠 2022. 1. 14.

30 : 쇼츠 2022. 5. 28.

31 : 쇼츠 2021. 7. 5.

32 : 쇼츠 2021. 9. 3.

33 : 쇼츠 2022. 4. 25.

34 : 쇼츠 2021. 8. 31.

35 : 쇼츠 20

In [602]:
day_list2 = [] #전처리 후 업로드일

#업로드일 날짜형식으로 통일화
for day in day_list : 
    day = day.replace('.', '')
    day = day.replace('최초 공개: ', '')
    day = day.replace('실시간 스트리밍 시작일: ', '')
    day = day.replace(' ', '-')
    day_list2.append(day)

#df화
youtubeDic = {
    '제목': name_list,
    '업로드일': day_list2,
    '주소': url_list,
    '해시태그' : hashtag_list
}

youtubeDf = pd.DataFrame(youtubeDic)

# datetime 타입으로 변환
youtubeDf['업로드일'] = pd.to_datetime(youtubeDf['업로드일'])

# 중복값제거
youtubeDf = youtubeDf.drop_duplicates(['주소'], keep='first')

youtubeDf.tail()

,제목,업로드일,주소,해시태그
148,2021년 남자들이 좋아하는 vs 극혐하는 여자 패션,2021-07-26,https://www.youtube.com/watch?v=fiXg1kivlOo,#인터뷰 #토크 #연애 #여자패션 #데이트페션
149,청스커트 리폼,2021-07-15,https://www.youtube.com/shorts/V1mUnemoZdk,
150,𝒇𝒂𝒍𝒍 𝒉𝒂𝒖𝒍👢가을 데일리룩 고민된다구요? 핏 & 색감 완벽스한 21가지 가을하...,2021-09-28,https://www.youtube.com/watch?v=FxLJeNEEQHw,#패션하울 #가을옷 #데일리룩
151,청바지 절대로 버리지 마세요! 청바지를 절대로 버리면 안되는 이유 / 리폼으로 변신...,2022-04-25,https://www.youtube.com/watch?v=dlKWRqBmXUk,#청바지리폼 #이너백만들기 #청치마리폼
152,여우꼬리의 옷장투어 세번째편 !! 간절기 패션 팁,2021-10-04,https://www.youtube.com/watch?v=o5P9XBJE9hw,


In [603]:
#postgresql 접속
conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

table = 'yt_clothes'

#table에 데이터 넣기
for i in range(len(youtubeDf)) :
    cur.execute(insert_command(table), (youtubeDf.iloc[i][0], youtubeDf.iloc[i][1], youtubeDf.iloc[i][2], youtubeDf.iloc[i][3]))
    conn.commit()

#db 연결 종료
conn.cursor().close()
conn.close()


In [572]:
#varchar(500) type에 안들어가면 실행
b = 0

for i in range(len(hashtag_list)) :
    a = len(hashtag_list[i])

    if a > b :
        b = a
        print(i, b)

print(hashtag_list[98][:203])

0 23
9 35
10 73
23 91
53 255
#진주백 #장미백 #체인백 #핑크뮬 #장미 #플라워 #장미꽃 #왕퍼프 #퍼프블라우스 #로즈 #로즈데이 #벚꽃 #꽃놀이 #꽃구경 #꽃나들이 #나들이 #나들이룩 #봄룩 #봄코디 #봄패션 #봄나들이 #여신룩 #여신 #롱치마 #롱스커트 #머메이드 #머메이드스커트 #맥시 #치마 #스커트 #맥시롱스커트 #하이웨스트 #꽃 #꽃길 #벚꽃길 #벚꽃엔딩 #하이웨스트스커트 #꾸안꾸


In [571]:
hashtag_list[98] = hashtag_list[98][:203]

# #############여기까지만#############

In [ ]:
#table 확인하기
cur.execute(select_command(table))
conn.commit()
cur.fetchall()

In [65]:
#db 연결 종료
conn.cursor().close()
conn.close()

create table yt_clothes (
	id serial,
	title varchar(100),
	date date,
	url varchar(200));